In [ ]:
%load_ext autoreload
%autoreload 2
%run notebook_setup.py

In [ ]:
import sys
import os
import pprint
from plotting import plt, plot_fills_long, create_forager_balance_figures
import pandas as pd
import numpy as np
import json
from config_utils import load_config, dump_config, format_config
from backtest import (
    prepare_hlcvs_mss,
    run_backtest,
    process_forager_fills,
    build_backtest_payload,
    execute_backtest,
    subset_backtest_payload,
)
from collections import defaultdict

pd.set_option("display.precision", 4)           # affects Series repr
pd.options.display.float_format = "{:.4f}".format  # consistent 4‑decimals everywhere


In [ ]:
config = load_config("configs/template.json")
# config["backtest"]["combine_ohlcvs"] = True
# config["backtest"]["start_date"] = "2023-03-01"
# config['backtest']['end_date'] = "2025-03-20"
# config["backtest"]["exchanges"] = ["binance", "bybit"]
exchange = "combined" if config["backtest"]["combine_ohlcvs"] else config["backtest"]["exchanges"][0]

In [ ]:
coins, hlcvs, mss, results_path, cache_dir, btc_usd_prices, timestamps = await prepare_hlcvs_mss(config, exchange)
config["backtest"]["coins"] = {exchange: coins}

In [ ]:
payload = build_backtest_payload(
    hlcvs=hlcvs,
    mss=mss,
    config=config,
    exchange=exchange,
    btc_usd_prices=btc_usd_prices,
    timestamps=timestamps,
)

In [ ]:
# config['bot']['long']['n_positions'] = 3
# config['bot']['long']['filter_rolling_window'] = 10
# config['bot']['long']['filter_relative_volume_clip_pct'] = 0.5
# config['bot']['short']['n_positions'] = 0.0
# config['backtest']['btc_collateral_cap'] = 0.7
#config['bot']['long']['close_trailing_threshold_pct'] = 0.004
#config['bot']['long']['close_trailing_retracement_pct'] = 0.004

In [ ]:
fills, equities, analysis = execute_backtest(payload, config)

In [ ]:
# Keep every other coin to probe subset performance
coin_indices = list(range(0, payload.bundle.coins_len(), 2))
subset_payload = subset_backtest_payload(payload, coin_indices=coin_indices)
subset_fills, subset_equities, subset_analysis = execute_backtest(subset_payload, config)
subset_analysis["adg_btc"]

The helper above makes it easy to carve out ad-hoc slices of the HLCV cube. You can also target specific tickers explicitly:

In [ ]:
fdf, analysis_py, bal_eq = process_forager_fills(
    subset_fills,
    config["backtest"]["coins"][exchange],
    subset_payload.bundle.hlcvs,
    subset_equities,
)
fdf.loc[:,'we'] = fdf.psize * fdf.pprice / fdf.usd_total_balance
delta = fdf.we - fdf.we.groupby(fdf["coin"]).shift(fill_value=0.0)
fdf.loc[:,"twe"] = delta.cumsum()
for k in analysis_py:
    if k not in subset_analysis:
        subset_analysis[k] = analysis_py[k]
display_keys = {
    'adg_btc',
    'adg_usd',
    'drawdown_worst_btc',
    'drawdown_worst_usd',
    'gain_btc',
    'gain_usd',
    "peak_recovery_hours_pnl",
    "peak_recovery_hours_equity_usd",
    "peak_recovery_hours_equity_btc",
    "sharpe_ratio_btc",
    "sharpe_ratio_usd",
    "position_held_hours_max",
    "position_unchanged_hours_max",
}
pprint.pprint({k: v for k, v in subset_analysis.items() if k in display_keys})
_ = create_forager_balance_figures(bal_eq, fast=True, stride=10, logy=True)

In [ ]:
focus_payload = subset_backtest_payload(
    payload,
    coin_symbols=["BTC/USDT:USDT", "ETH/USDT:USDT", "XRP/USDT:USDT", "ADA/USDT:USDT", "SOL/USDT:USDT"],
)
focus_fills, focus_equities, focus_analysis = execute_backtest(focus_payload, config)
focus_analysis["adg_btc"]

In [ ]:
fdf, analysis_py, bal_eq = process_forager_fills(
    focus_fills,
    config["backtest"]["coins"][exchange],
    focus_payload.bundle.hlcvs,
    focus_equities,
)
fdf.loc[:,'we'] = fdf.psize * fdf.pprice / fdf.usd_total_balance
delta = fdf.we - fdf.we.groupby(fdf["coin"]).shift(fill_value=0.0)
fdf.loc[:,"twe"] = delta.cumsum()
for k in analysis_py:
    if k not in focus_analysis:
        focus_analysis[k] = analysis_py[k]
display_keys = {
    'adg_btc',
    'adg_usd',
    'drawdown_worst_btc',
    'drawdown_worst_usd',
    'gain_btc',
    'gain_usd',
    "peak_recovery_hours_pnl",
    "peak_recovery_hours_equity_usd",
    "peak_recovery_hours_equity_btc",
    "sharpe_ratio_btc",
    "sharpe_ratio_usd",
    "position_held_hours_max",
    "position_unchanged_hours_max",
}
pprint.pprint({k: v for k, v in focus_analysis.items() if k in display_keys})
_ = create_forager_balance_figures(bal_eq, fast=True, stride=10, logy=True)

In [ ]:
payload.bundle.hlcvs

In [ ]:
fdf, analysis_py, bal_eq = process_forager_fills(
    fills,
    config["backtest"]["coins"][exchange],
    hlcvs,
    equities,
)
fdf.loc[:,'we'] = fdf.psize * fdf.pprice / fdf.usd_total_balance
delta = fdf.we - fdf.we.groupby(fdf["coin"]).shift(fill_value=0.0)
fdf.loc[:,"twe"] = delta.cumsum()
for k in analysis_py:
    if k not in analysis:
        analysis[k] = analysis_py[k]
display_keys = {
    'adg_btc',
    'adg_usd',
    'drawdown_worst_btc',
    'drawdown_worst_usd',
    'gain_btc',
    'gain_usd',
    "peak_recovery_hours_pnl",
    "peak_recovery_hours_equity_usd",
    "peak_recovery_hours_equity_btc",
    "sharpe_ratio_btc",
    "sharpe_ratio_usd",
    "position_held_hours_max",
    "position_unchanged_hours_max",
}
pprint.pprint({k: v for k, v in analysis.items() if k in display_keys})
_ = create_forager_balance_figures(bal_eq, fast=True, stride=10, logy=True)

In [ ]:
fdf.twe.plot()

In [ ]:
opened_pos = defaultdict(float)
durations = []
for row in fdf.itertuples():
    if row.coin in opened_pos:
        if row.psize == 0.0:
            durations.append((row.coin, row.timestamp.timestamp() - opened_pos[row.coin]))
            del opened_pos[row.coin]
    else:
        if row.psize != 0.0:
            opened_pos[row.coin] = row.timestamp.timestamp()
max_durations = defaultdict(float)
for x in durations:
    max_durations[x[0]] = max(x[1], max_durations[x[0]])
sorted(max_durations.items(), key=lambda x: x[1])

In [ ]:
coins_sorted_by_volume = fdf.groupby("coin").fee_paid.sum().sort_values().index.to_list()
coin = coins_sorted_by_volume[0]
coin = 'APT'
print(coin)
cdf, fdfc = plot_fills_long(hlcvs, fdf, coins, coin, start_pct=0.0, end_pct=1.0)

In [ ]:
json.dumps(config['live']['approved_coins']['long'][1::2])

In [ ]:
# performers worst to best
for x in fdf.groupby("coin").pnl.sum().sort_values().to_dict().items():
    print(x)